# Algorithmes de recherche non informée


Les algorithmes que nous allons implementer dans ce TD serons utilisés pour resoudre les problèmes formalisés dans le TD3 précedent. Donc nous devons d'abord charger le notebook TD3-solution.ipynb précédent:  

In [18]:
import sys

In [1]:
%%capture
%run TD3-solution.ipynb

La fonction squelette d'un algorithme de recherche graphique est :

```
function  GRAPH-SEARCH( problem, frontier) returns a solution, or failure 
         initialize the frontier using the initial state of problem
         initialize the explored set to empty  
         loop do
                if the frontier is empty then return failure
                choose a leaf node and remove it from the frontier
                if the node contains a goal state then 
                         return the corresponding solution 
                add the node  explored set
                expand the chosen node, adding the resulting nodes to the frontier 
```

La recherche graphique  est basée sur la notion de noeud dont l'implémentation est donnée dans la cellule suivante:

In [2]:

        
class Node:
    """A node in a search tree. Contains a pointer to the parent (the node
    that this is a successor of) and to the actual state for this node. Note
    that if a state is arrived at by two paths, then there are two nodes with
    the same state. Also includes the action that got us to this state, and
    the total path_cost (also known as g) to reach the node. Other functions
    may add an f and h value; see best_first_graph_search and astar_search for
    an explanation of how the f and h values are handled. You will not need to
    subclass this class."""

    def __init__(self, state, parent=None, action=None, path_cost=0):
        """Create a search tree Node, derived from a parent by an action."""
        self.state = state
        self.parent = parent
        self.action = action
        self.path_cost = path_cost
        self.depth = 0
        if parent:
            self.depth = parent.depth + 1

    def __repr__(self):
        return "<Node {}>".format(self.state)

    def __lt__(self, node):
        return self.state < node.state

    def expand(self, problem):
        """List the nodes reachable in one step from this node."""
        return [self.child_node(problem, action)
                for action in problem.actions(self.state)]

    def child_node(self, problem, action):
        """[Figure 3.10]"""
        next_state = problem.get_successors(self.state, action)
        next_node = Node(next_state, self, action, problem.path_cost(self.path_cost, self.state, action, next_state))
        return next_node

    def solution(self):
        """Return the sequence of actions to go from the root to this node."""
        return [node.action for node in self.path()[1:]]

    def path(self):
        """Return a list of nodes forming the path from the root to this node."""
        node, path_back = self, []
        while node:
            path_back.append(node)
            node = node.parent
        return list(reversed(path_back))

    # We want for a queue of nodes in breadth_first_graph_search or
    # astar_search to have no duplicated states, so we treat nodes
    # with the same state as equal. [Problem: this may not be what you
    # want in other contexts.]

    def __eq__(self, other):
        return isinstance(other, Node) and self.state == other.state

    def __hash__(self):
        # We use the hash value of the state
        # stored in the node instead of the node
        # object itself to quickly search a node
        # with the same state in a Hash Table
        return hash(self.state)


### 1. Algorithme de recherche en largeur d'abord (Beardth first search)

In [3]:

# In this algorithm we use FIFO Queue then we must import deque from pyhton collections modul
from collections import deque

def breadth_first_graph_search(problem):
    """ 
    Note that this function can be implemented in a
    single line as below:
    return graph_search(problem, FIFOQueue())
    where graph_search is the implementation of the algorithm above
    """
    node = Node(problem.initial)
    if problem.is_goal_state(node.state): 
        return node
    frontier = deque([node])
    explored = set()
    while frontier:
        node = frontier.popleft()  
        explored.add(node.state)
        for child in node.expand(problem):
            if child.state not in explored and child not in frontier:
                if problem.is_goal_state(child.state):
                    return child
                frontier.append(child)
    return None





In [4]:
# Tester la résoulution les problèmes du TD3 par cet algorithme
# Votre code ici
goal_node = breadth_first_graph_search(world_block_problem)

for n in goal_node.path():
    print(n.state)
    
 

((2, 5, 3), (1, 4), (), (), ())
((2, 5), (1, 4), (3,), (), ())
((2,), (1, 4), (3,), (5,), ())
((2,), (1,), (3,), (5, 4), ())
((2,), (1,), (), (5, 4, 3), ())
((), (1,), (), (5, 4, 3, 2), ())
((), (), (), (5, 4, 3, 2, 1), ())


### 2. Algorithme de recherche en profondeur d'abord (Depth first search)

In [5]:
# Votre code ici
def depth_first_graph_search(problem):
    node = Node(problem.initial)
    if problem.is_goal_state(node.state): 
        return node
    frontier = list([node])
    explored = set()
    while frontier:
        node = frontier.pop()  
        explored.add(node.state)
        for child in node.expand(problem):
            if child.state not in explored and child not in frontier:
                if problem.is_goal_state(child.state):
                    return child
                frontier.append(child)
    return None

In [6]:
# Tester la résoulution les problèmes du TD3 par cet algorithme
# Votre code ici
goal_node = depth_first_graph_search(world_block_problem)

for n in goal_node.path():
    print(n.state)



((2, 5, 3), (1, 4), (), (), ())
((2, 5, 3), (1,), (4,), (), ())
((2, 5, 3), (1,), (), (4,), ())
((2, 5, 3), (), (), (4, 1), ())
((2, 5), (), (), (4, 1, 3), ())
((2,), (), (), (4, 1, 3, 5), ())
((), (2,), (), (4, 1, 3, 5), ())
((), (2, 5), (), (4, 1, 3), ())
((), (2, 5, 3), (), (4, 1), ())
((), (2, 5, 3, 1), (), (4,), ())
((4,), (2, 5, 3, 1), (), (), ())
((4,), (2, 5, 3), (1,), (), ())
((4,), (2, 5, 3), (), (1,), ())
((4,), (2, 5), (), (1, 3), ())
((4,), (2,), (), (1, 3, 5), ())
((4,), (), (), (1, 3, 5, 2), ())
((), (4,), (), (1, 3, 5, 2), ())
((), (4, 2), (), (1, 3, 5), ())
((), (4, 2, 5), (), (1, 3), ())
((), (4, 2, 5, 3), (), (1,), ())
((1,), (4, 2, 5, 3), (), (), ())
((1,), (4, 2, 5), (3,), (), ())
((1,), (4, 2, 5), (), (3,), ())
((1,), (4, 2), (), (3, 5), ())
((1,), (4,), (), (3, 5, 2), ())
((1,), (), (), (3, 5, 2, 4), ())
((), (1,), (), (3, 5, 2, 4), ())
((), (1, 4), (), (3, 5, 2), ())
((), (1, 4, 2), (), (3, 5), ())
((), (1, 4, 2, 5), (), (3,), ())
((3,), (1, 4, 2, 5), (), (), ()

### 3. Algorithme de recherche en profondeur limité (Limited Depth Search)

In [13]:
# Votre code ici
def limited_depth_graph_search(problem,limit):
    node = Node(problem.initial)
    if problem.is_goal_state(node.state): 
        return node
    frontier = list([node])
    explored = set()
    while frontier:
        node = frontier.pop()
        if node.depth >= limit:
            continue
        explored.add(node.state)
        for child in node.expand(problem):
            if child.state not in explored and child not in frontier:
                if problem.is_goal_state(child.state):
                    return child
                frontier.append(child)
    return None

In [14]:
# Tester la résoulution les problèmes du TD3 par cet algorithme
# Votre code ici
goal_node_1 = limited_depth_graph_search(world_block_problem,6)
goal_node_2 = limited_depth_graph_search(world_block_problem,7)

if goal_node_1 != None:
    for n in goal_node.path():
        print(n.state)
print("|||||||||||||||||||||||||||||")
if goal_node_2 != None:
    for n in goal_node.path():
        print(n.state)



|||||||||||||||||||||||||||||
((2, 5, 3), (1, 4), (), (), ())
((2, 5, 3), (1,), (4,), (), ())
((2, 5), (1,), (4,), (3,), ())
((2,), (1,), (4,), (3,), (5,))
((2,), (1,), (), (3,), (5, 4))
((2,), (1,), (), (), (5, 4, 3))
((), (1,), (), (), (5, 4, 3, 2))
((), (), (), (), (5, 4, 3, 2, 1))


### 4. Algorithme de recherche d'approfondissement itérative (iterative deepening search)

In [9]:
# Votre code ici

In [21]:
def iterative_deepening_graph_search(problem):
    for depth in range(sys.maxsize):
        node = limited_depth_graph_search(problem, depth)
        if node != None:
            print(node.state)
            return
            

In [22]:
# Tester la résoulution les problèmes du TD3 par cet algorithme
# Votre code ici

goal_node = iterative_deepening_graph_search(world_block_problem)
goal_node

((), (), (), (), (5, 4, 3, 2, 1))
